In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [2]:
import kagglehub

# Download the dataset to a local directory
path = kagglehub.dataset_download("salahuddinahmedshuvo/grocery-inventory-and-sales-dataset")

print("Path to dataset files:", path)

c:\Users\Nikunj\miniconda3\envs\tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Nikunj\.cache\kagglehub\datasets\salahuddinahmedshuvo\grocery-inventory-and-sales-dataset\versions\1


In [3]:
import os

path = r"C:\Users\Nikunj\.cache\kagglehub\datasets\salahuddinahmedshuvo\grocery-inventory-and-sales-dataset\versions\1"
print(os.listdir(path))

['Grocery_Inventory_and_Sales_Dataset.csv']


In [4]:
import pandas as pd

file_path = r"C:\Users\Nikunj\.cache\kagglehub\datasets\salahuddinahmedshuvo\grocery-inventory-and-sales-dataset\versions\1\Grocery_Inventory_and_Sales_Dataset.csv"
df = pd.read_csv(file_path)

print(df.head())
print(df.columns)

    Product_ID     Product_Name             Catagory  Supplier_ID  \
0  29-205-1132       Sushi Rice      Grains & Pulses  38-037-1699   
1  40-681-9981   Arabica Coffee            Beverages  54-470-2479   
2  06-955-3428       Black Rice      Grains & Pulses  54-031-2945   
3  71-594-6552  Long Grain Rice      Grains & Pulses  63-492-7603   
4  57-437-1828             Plum  Fruits & Vegetables  54-226-4308   

  Supplier_Name  Stock_Quantity  Reorder_Level  Reorder_Quantity Unit_Price  \
0     Jaxnation              22             72                70     $4.50    
1       Feedmix              45             77                 2    $20.00    
2        Vinder              30             38                83     $6.00    
3    Brightbean              12             59                62     $1.50    
4    Topicstorm              37             30                74     $4.00    

  Date_Received Last_Order_Date Expiration_Date      Warehouse_Location  \
0     8/16/2024       6/29/2024    

In [5]:
data = pd.DataFrame(df)
data.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/29/2024,5/8/2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,6/10/2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/11/2024,10/5/2024,15068 Scoville Court,62,25,Backordered


In [11]:
# Reorder Margin: How far stock is from reorder threshold
data['Reorder_Margin'] = data['Stock_Quantity'] - data['Reorder_Level']

# Sales per Stock: Normalize sales relative to stock quantity
data['Sales_per_Stock'] = data['Sales_Volume'] / (data['Stock_Quantity'] + 1)  # +1 to avoid division by zero

# Days Since Received: Time difference between the latest date and Date_Received
data['Date_Received'] = pd.to_datetime(data['Date_Received'], errors='coerce')
latest_date = data['Date_Received'].max()
data['Days_Since_Received'] = (latest_date - data['Date_Received']).dt.days


# Features
numerical = [
    'Stock_Quantity', 'Reorder_Level', 'Unit_Price',
    'Inventory_Turnover_Rate', 'Shelf_Life_Days',
    'Reorder_Margin', 'Sales_per_Stock', 'Days_Since_Received'
]

categorical = ['Catagory', 'Warehouse_Location', 'Supplier_Name']

# One-hot encode
data_encoded = pd.get_dummies(data[categorical], drop_first=True)

# Final input and output
X = pd.concat([data[numerical], data_encoded], axis=1)
y = data['Sales_Volume']

# Scale numerical features
scaler = StandardScaler()
X[numerical] = scaler.fit_transform(X[numerical])


In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n🌲 Fold {fold+1}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    rf = RandomForestRegressor(
        n_estimators=2000,
        max_depth=30,
        max_features='sqrt',
        min_samples_split = 20,
        random_state=42
    )
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_val)
    val_mae = mean_absolute_error(y_val, y_pred)

    print(f"✅ MAE: {val_mae:.2f}")
    mae_scores.append(val_mae)

print("\n📊 Tuned Random Forest Avg MAE:", np.mean(mae_scores))


🌲 Fold 1
✅ MAE: 16.49

🌲 Fold 2
✅ MAE: 15.95

🌲 Fold 3
✅ MAE: 15.33

🌲 Fold 4
✅ MAE: 15.87

🌲 Fold 5
✅ MAE: 15.68

📊 Tuned Random Forest Avg MAE: 15.866284139321582


In [20]:
expected_cols = X.columns.tolist()


In [21]:
new_input = pd.DataFrame([{
    'Stock_Quantity': 40,
    'Reorder_Level': 20,
    'Unit_Price': 25.0,
    'Inventory_Turnover_Rate': 5,
    'Shelf_Life_Days': 60,
    'Reorder_Margin': 20,
    'Sales_per_Stock': 1.2,
    'Days_Since_Received': 15,
    # dummy columns (set 1 or 0 accordingly)
    'Catagory_Beverages': 1,
    'Warehouse_Location_A1': 1,
    'Supplier_Name_ABC Suppliers': 1
}])


In [22]:
# Add missing dummy columns as 0
for col in expected_cols:
    if col not in new_input.columns:
        new_input[col] = 0

# Ensure column order matches
new_input = new_input[expected_cols]


C:\Users\Nikunj\AppData\Local\Temp\ipykernel_25448\2631757330.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_input[col] = 0
C:\Users\Nikunj\AppData\Local\Temp\ipykernel_25448\2631757330.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_input[col] = 0
C:\Users\Nikunj\AppData\Local\Temp\ipykernel_25448\2631757330.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

In [23]:
new_input[numerical] = scaler.transform(new_input[numerical])
predicted_sales = rf.predict(new_input)
print("📈 Predicted Sales Volume:", predicted_sales[0])


📈 Predicted Sales Volume: 60.90207417113678


In [29]:
import pickle

with open("final_model.pkl", "wb") as f:
    pickle.dump(rf,f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler,f)

with open("expected_cols.pkl", "wb") as f:
    pickle.dump(expected_cols,f)
